In [552]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt
import panel as pn
pn.extension(comms='vscode')
%matplotlib inline

mode = 'bid_price'
#mode = 'acc_rej'
assert mode in ['acc_rej','bid_price']
total_rounds = 2
max_cap = 5
max_horizon = 40

stationary_arrival_rate = 0.3

peak = 0.61538
non_stationary_arrival_rates = np.concatenate((np.linspace(0,peak,10), np.linspace(peak,0,31)[1:]))
non_stationary_arrival_rates.mean()

# buttons
button_size_small = 50
button_size_large = 100

class DataHandler():

    def __init__(self,max_cap,max_horizon,total_rounds,user_id,mode):
        self.total_revenue = 0
        self.current_wtp = 0

        self.current_occ = 0 
        self.current_time = 0

        self.max_horizon = max_horizon
        self.max_cap = max_cap

        self.wtps = np.random.randint(50, 101, size=self.max_horizon)

        self.current_round = 1
        self.total_rounds = total_rounds

        self.id = user_id
        self.mode = mode
        self.log = pd.DataFrame(columns=['mode','user_id','round','timestep','wtp','bid_price','accept','revenue'])

    def reset(self):
        self.total_revenue = 0
        self.current_wtp = 0

        self.current_occ = 0 
        self.current_time = 0

        self.wtps = np.random.randint(50, 101, size=self.max_horizon)

        self.current_round += 1


    @property
    def leftover_capacity(self):
        return self.max_cap - self.current_occ

    @property
    def leftover_time(self):
        return self.max_horizon - self.current_time

data = DataHandler(max_cap,max_horizon,total_rounds,'leif',mode)

In [553]:
header = pn.pane.Markdown("# Panel Dashboarding")
task_descr = pn.pane.Markdown("""
###Accept or Reject arriving Customers<br>
You start managing customers when there are {} days left until departure.<br>
Each day, at most one customer requests a seat with given probability.<br>
The customer indicates a maximum price he is willing to pay.""".format(max_horizon))

In [554]:
app = pn.Column(header, task_descr)

In [555]:
round = pn.pane.Markdown("## Round [{} / {}]".format(data.current_round,data.total_rounds))
app.append(round)

In [556]:
total_revenue = pn.pane.Markdown("##Total Revenue: {} €".format(data.total_revenue))
app.append(total_revenue)

In [557]:
# leftover capacity bar
leftover_capacity = pn.pane.Markdown(f"### Leftover Capacity: {data.leftover_capacity}")
leftover_capacity_bar = pn.indicators.Progress(name='Progress', value=0,max=data.max_cap, width=200)
leftover_element = pn.Column(leftover_capacity, leftover_capacity_bar)


# days until departure bar
days_until_departure = pn.pane.Markdown(f"### Days Until Departure: {data.leftover_time}")
days_until_departure_bar = pn.indicators.Progress(name='Progress', value=0,max=data.max_horizon, width=200)
days_until_departure_element = pn.Column(days_until_departure, days_until_departure_bar)

bars = pn.Row(leftover_element, days_until_departure_element)

progress = pn.indicators.Progress(name='Progress', value=20, width=200)

app.append(bars)


In [558]:
### ACCEPT REJECT BLOCK


customer_information = pn.pane.Markdown("### A new customer arrives and is willing to pay {}".format(data.wtps[0]),width=350,height=80)

accept_button = pn.widgets.Button(name='Accept', button_type='success',width=button_size_small)
reject_button = pn.widgets.Button(name='Reject', button_type='danger',width=button_size_small)
ok_button = pn.widgets.Button(name='OK', button_type='primary',width=button_size_large)
next_round_button = pn.widgets.Button(name='Next Round', button_type='primary',width=button_size_large)
button_row = pn.Row(accept_button, reject_button)

accept_reject_block = pn.Column(customer_information, button_row)

### BID PRICE BLOCK

bid_price = pn.widgets.IntSlider(name='Bid Price', start=0, end=100, value=0, step=1)
bid_price_button = pn.widgets.Button(name='Execute Bid Price', button_type='primary',disabled=True,width=button_size_large)
bid_price_information = pn.pane.Markdown("### Set a bid price for the first day.<br> ",width=350,height=80)
bid_price_block = pn.Column(bid_price_information, bid_price, bid_price_button)

if mode == 'acc_rej':
    app.append(accept_reject_block)
elif mode == 'bid_price':
    app.append(bid_price_block)

# observe changes in bid_price int slider
def bid_price_change(event):
    if bid_price.value > 0:
        bid_price_button.disabled = False
    else:
        bid_price_button.disabled = True

bid_price.param.watch(bid_price_change, 'value')



def exec_bid_price(event):
    deal = bid_price.value <= data.wtps[data.current_time]

    if deal:
        data.total_revenue += bid_price.value  #incr revenue
        data.current_occ += 1                  #incr occ

        bid_price_information.object = f"### You sold a seat for {bid_price.value}€.<br>Set the bid price for the next day."

    else:
        bid_price_information.object = f"### You did not sell a seat.<br>Set the bid price for the next day."
    
    data.current_time += 1                 #move in time
    leftover_capacity.object = f"### Leftover Capacity: {data.leftover_capacity}"
    leftover_capacity_bar.value = data.current_occ
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"
    days_until_departure_bar.value = data.leftover_time
    total_revenue.object = "##Total Revenue: {} €".format(data.total_revenue)

    data.log = data.log.append({'mode':data.mode,'user_id': data.id, 'round': data.current_round, 
    'timestep': data.current_time, 'wtp': data.current_wtp,'bid_price': bid_price.value,
    'accept': int(deal), 'revenue': data.total_revenue}, ignore_index=True)

    if data.leftover_capacity == 0 or data.leftover_time == 0:

        if data.current_round < data.total_rounds:
            bid_price_block.remove(bid_price_button)
            bid_price_block.append(next_round_button)
            bid_price_information.object = "###The flight is fully booked or departing today.<br> "

        else:
            bid_price_block.remove(bid_price_button)
            bid_price_block.remove(bid_price)
            bid_price_information.object = """
            ###The flight is fully booked or departing today.
            Your data has been saved. Thank you for participating. You can close this window now."""

In [559]:



# when clicking accept button, update the progress bar
def accept(event):
    """
    Accept a customer with a given wtp.
    Update all elements of the data handler
    and the progress bar.
    Then execute next_day() to jump to the next day and manage rounds.
    """
    

    data.current_occ +=1
    data.current_time +=1
    
    leftover_capacity_bar.value = data.current_occ
    days_until_departure_bar.value = data.current_time

    leftover_capacity.value = f"### Leftover Capacity: {data.leftover_capacity}"
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"

    data.total_revenue += data.current_wtp
    total_revenue.object = "##Total Revenue: {} €".format(data.total_revenue)

    data.log = data.log.append({'mode':data.mode,'user_id': data.id, 'round': data.current_round, 
    'timestep': data.current_time, 'wtp': data.current_wtp,'bid_price':data.current_wtp,
    'accept': 1, 'revenue': data.total_revenue}, ignore_index=True)

    new_day()

def reject(event):
    """
    Reject a customer with a given wtp.
    Update all elements of the data handler
    and the progress bar.
    Then execute next_day() to jump to the next day and manage rounds.
    """
    data.current_time +=1
    days_until_departure_bar.value = data.current_time
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"

    data.log = data.log.append({'mode':data.mode,'user_id': data.id, 'round': data.current_round, 
    'timestep': data.current_time, 'wtp': data.current_wtp,'bid_price':data.current_wtp + 1,
    'accept': 0, 'revenue': data.total_revenue}, ignore_index=True)
    
    new_day()
    
def new_day(event=None):
    """
    Check if the current round is over.
    If yes, reset the data handler and update the round counter.
    In any case, update the wtp and the customer information.
    """

    data.current_wtp = 0
    if data.leftover_capacity == 0 or data.leftover_time == 0:

        if data.current_round < data.total_rounds:
            button_row.clear()
            button_row.append(next_round_button)
            customer_information.object = "###The flight is fully booked or departing today."

        else:
            button_row.clear()
            customer_information.object = """
            ###The flight is fully booked or departing today.
            Your data has been saved. Thank you for participating. You can close this window now."""


    else:
        customer_arrives = np.random.choice([True, False], p=[stationary_arrival_rate, 1-stationary_arrival_rate])
        if customer_arrives:
            data.current_wtp = data.wtps[data.current_time]
            customer_information.object = "### A new customer arrives and is willing to pay {}".format(data.current_wtp)
            button_row.clear()
            button_row.append(accept_button)
            button_row.append(reject_button)

        else:
            customer_information.object = "### No customer arrives today"
            button_row.clear()
            button_row.append(ok_button)

def reset(event=None):
    """
    Reset the data handler and update the round counter.
    """

    data.reset()

    round.object = "## Round [{} / {}]".format(data.current_round,data.total_rounds)

    total_revenue.object = "##Total Revenue: {} €".format(data.total_revenue)

    leftover_capacity_bar.value = data.current_occ
    days_until_departure_bar.value = data.current_time

    leftover_capacity.value = f"### Leftover Capacity: {data.leftover_capacity}"
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"

    if data.mode == 'acc_rej':
        button_row.clear()
        button_row.append(accept_button)
        button_row.append(reject_button)
        new_day()

    else:
        bid_price_block.remove(next_round_button)
        bid_price_block.append(bid_price_button)
        bid_price_information.object = "### Set a bid price for the first day.<br> "
        
next_round_button.on_click(reset)
accept_button.on_click(accept)
reject_button.on_click(reject)
ok_button.on_click(reject)

bid_price_button.on_click(exec_bid_price)



Watcher(inst=Button(button_type='primary', disabled=True, name='Execute Bid Price', width=100), cls=<class 'panel.widgets.button.Button'>, fn=<function exec_bid_price at 0x283025a60>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [560]:
# password field
user_field = pn.widgets.TextInput(name='User ID', placeholder='Enter your user ID')
password_field = pn.widgets.PasswordInput(name='Enter your password', placeholder='your password...')
master_container = pn.Column(user_field,password_field)

correct_passwords = ['1234']

def check_password(event):
    """
    Check if the entered password is correct.
    If yes, show the experiment.
    If not, show an error message.
    """
    pw_check = password_field.value == user_field.value + '_manager'

    if pw_check:
        master_container.remove(password_field)
        master_container.remove(user_field)
        master_container.append(app)
        data.id = user_field.value
    else:
        password_field.name = 'Wrong password. Try again.'
        password_field.value = ''

password_field.param.watch(check_password, 'value')

Watcher(inst=PasswordInput(name='Enter your password', placeholder='your password...'), cls=<class 'panel.widgets.input.PasswordInput'>, fn=<function check_password at 0x28324bca0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [561]:
master_container.show()

Launching server at http://localhost:56108


In [562]:
# from bokeh.plotting import figure

# p1 = figure(width=300, height=300)
# p1.line([1, 2, 3], [1, 2, 3])

# tabs = pn.Tabs(p1)

# # Add a tab
# tabs.append(('Slider', pn.widgets.FloatSlider()))

# # Add multiple tabs
# tabs.extend([
#     ('Text', pn.widgets.TextInput()),
#     ('Color', pn.widgets.ColorPicker())
# ])

# tabs